# MODEL TRAINING - testing the full approach - DREAMS
saving models in a different folder using pickle, saving them in the dataframe causes xgboost to crash

**DO NOT RUN AGAIN**

In [1]:
#external libraries
import numpy as np
import os
import matplotlib.pyplot as plt
import matplotlib.colors as clt
import plotly
import plotly.subplots as sb
import plotly.express as px
import plotly.graph_objects as go
import dotenv
import pandas as pd
import scipy.fft as fft
import scipy.signal as sg
import scipy.io as sio
import pickle as pkl
import xgboost as xgb
import time
import uuid

#project library
from spinco import *

#environment variables
dotenv.load_dotenv('lab.env')

#project variables
datapath=os.environ['DATAPATH']
cognipath=datapath+"/COGNITION"
dreamspath=datapath+"/DREAMS"
masspath=datapath+"/MASS"


## experiment id
we'll use it to create a folder at the end of the script

In [2]:
experimentId=str(uuid.uuid4())

## define a fixed samplerate

In [3]:
samplerate=200

## load dreams

In [5]:
signals, annotations, signalsMetadata = loadDREAMSSpindles(dreamspath)

SubjectId: 0001--------------
resampling from 100 to 200
SubjectId: 0003--------------
resampling from 50 to 200
SubjectId: 0006--------------
duration discrepancy, removing last 0.7 seconds


In [6]:
annotations=annotations[annotations.type=='spindle'].reset_index(drop=True)
len(annotations)

764

In [7]:
minDuration=0.3
maxDuration=2.9
annotations=annotations[annotations.duration>minDuration].reset_index(drop=True)
annotations=annotations[annotations.duration<maxDuration].reset_index(drop=True)
len(annotations)

764

In [8]:
annotations.head()

,index,startTime,duration,channel,subjectId,labelerId,type,samplerate,stopTime,startInd,stopInd
0,0,282.24,0.72,C3-A1,0001,0001,spindle,200,282.96,56448,56592
1,1,311.72,1.54,C3-A1,0001,0001,spindle,200,313.26,62344,62652
2,2,340.28,0.72,C3-A1,0001,0001,spindle,200,341.00,68056,68200
3,3,366.83,0.65,C3-A1,0001,0001,spindle,200,367.48,73366,73496
4,4,373.74,0.50,C3-A1,0001,0001,spindle,200,374.24,74748,74848


In [9]:
signalsMetadata.head()

,filename,channel,subjectId,duration,samplerate,database
0,excerpt1.txt,C3-A1,0001,1800,200,DREAMS
1,excerpt2.txt,CZ-A1,0002,1800,200,DREAMS
2,excerpt3.txt,C3-A1,0003,1800,200,DREAMS
3,excerpt4.txt,CZ-A1,0004,1800,200,DREAMS
4,excerpt5.txt,CZ-A1,0005,1800,200,DREAMS


## define a fixed feature selection to use

In [10]:
featureSelection=loadPickle("featureSelection26.pkl")
featureSelection

,window,characteristic,bandName,score
0,2,hjortActivity,beta2,69.0
1,1,hjortActivity,sigma,63.0
2,2,hjortActivity,beta1,42.0
3,2,hjortActivity,delta2,41.0
4,1.5,sigmaIndex,broadband,39.0
5,2,sigmaIndex,broadband,39.0
6,2,hjortActivity,theta,38.0
7,2,hjortMobility,beta1,37.0
8,2,hjortActivity,alpha,35.0
9,1,sigmaIndex,broadband,34.0


## load data split
10 fold CV with 4 subjects for testing

In [11]:
#by now we used fixed CV data splits instead
dataSplits=loadPickle("dataSplits_LOOCV.pkl")
dataSplits

,train,val,test
0,"[0002, 0003, 0005, 0006, 0007]","[0008, 0004]",0001
1,"[0003, 0005, 0006, 0007, 0008]","[0001, 0004]",0002
2,"[0001, 0002, 0004, 0005, 0006]","[0008, 0007]",0003
3,"[0001, 0002, 0003, 0005, 0008]","[0007, 0006]",0004
4,"[0001, 0003, 0006, 0007, 0008]","[0002, 0004]",0005
5,"[0001, 0002, 0004, 0005, 0008]","[0003, 0007]",0006
6,"[0001, 0002, 0003, 0006, 0008]","[0004, 0005]",0007
7,"[0001, 0002, 0004, 0005, 0006]","[0003, 0007]",0008


## define annotation criteria
we want to test the models in three diferent ways

In [15]:
annotationCriteria=pd.DataFrame({
    'criteriumId':['0001','0002','0003'],
    'criteriumName':['E1','E2','union'],
    'labelerIdList':[['0001'],['0002'],['0001','0002']]
})

In [16]:
annotationCriteria

,criteriumId,criteriumName,labelerIdList
0,0001,E1,[0001]
1,0002,E2,[0002]
2,0003,union,"[0001, 0002]"


In [17]:
experimentModels=pd.merge(annotationCriteria,dataSplits,how='cross')

In [18]:
experimentModels

,criteriumId,criteriumName,labelerIdList,train,val,test
0,0001,E1,[0001],"[0002, 0003, 0005, 0006, 0007]","[0008, 0004]",0001
1,0001,E1,[0001],"[0003, 0005, 0006, 0007, 0008]","[0001, 0004]",0002
2,0001,E1,[0001],"[0001, 0002, 0004, 0005, 0006]","[0008, 0007]",0003
3,0001,E1,[0001],"[0001, 0002, 0003, 0005, 0008]","[0007, 0006]",0004
4,0001,E1,[0001],"[0001, 0003, 0006, 0007, 0008]","[0002, 0004]",0005
5,0001,E1,[0001],"[0001, 0002, 0004, 0005, 0008]","[0003, 0007]",0006
6,0001,E1,[0001],"[0001, 0002, 0003, 0006, 0008]","[0004, 0005]",0007
7,0001,E1,[0001],"[0001, 0002, 0004, 0005, 0006]","[0003, 0007]",0008
8,0002,E2,[0002],"[0002, 0003, 0005, 0006, 0007]","[0008, 0004]",0001
9,0002,E2,[0002],"[0003, 0005, 0006, 0007, 0008]","[0001, 0004]",0002


## model fitting

In [19]:
n_jobs=4 # paralelization parameter for xgboost (keep it 8 as maximum)
learning_rate=0.4
subsample=0.6

num_boost_round=60  #number of boosting rounds per model

models=[]
modelIds=[]
valF1s=[]
spindleTimeRates=[]
for index, row in experimentModels.iterrows():
    print('**********************')
    print(str(index+1)+' of '+str(len(experimentModels)))
    #Define annotations criterium
    usedAnnotations=annotations[annotations.labelerId.isin(row.labelerIdList)].reset_index(drop=True)
    #Features
    trainFeatures=loadFeatureMatrix(row.train,featureSelection,signalsMetadata,samplerate,datapath)
    #Labels
    trainLabels=loadLabelsVector(row.train,usedAnnotations,signalsMetadata,samplerate)
    
    #Train the models
    params={
        'n_jobs':n_jobs,
        'learning_rate':learning_rate,
        'subsample':subsample,
        'objective':'binary:logistic'
    }
    trainDMatrix=xgb.DMatrix(data=trainFeatures,label=trainLabels)
    xgb_model = xgb.train(params,trainDMatrix,num_boost_round=num_boost_round)
    spindleTimeRates.append(sum(trainLabels)/len(trainLabels))
    models.append(xgb_model)
    modelIds.append(str(uuid.uuid4()))
        

**********************
1 of 24
**********************
2 of 24
**********************
3 of 24
**********************
4 of 24
**********************
5 of 24
**********************
6 of 24
**********************
7 of 24
**********************
8 of 24
**********************
9 of 24
**********************
10 of 24
**********************
11 of 24
**********************
12 of 24
**********************
13 of 24
**********************
14 of 24
**********************
15 of 24
**********************
16 of 24
**********************
17 of 24
**********************
18 of 24
**********************
19 of 24
**********************
20 of 24
**********************
21 of 24
**********************
22 of 24
**********************
23 of 24
**********************
24 of 24


In [20]:
experimentModels["modelId"]=modelIds
experimentModels["spindleTimeRate"]=spindleTimeRates

In [21]:
experimentModels

,criteriumId,criteriumName,labelerIdList,train,val,test,modelId,spindleTimeRate
0,0001,E1,[0001],"[0002, 0003, 0005, 0006, 0007]","[0008, 0004]",0001,70cf34de-3c08-4b43-8ead-e9c6231c6447,0.017738
1,0001,E1,[0001],"[0003, 0005, 0006, 0007, 0008]","[0001, 0004]",0002,417d5651-7922-417e-b915-f19589634c82,0.017258
2,0001,E1,[0001],"[0001, 0002, 0004, 0005, 0006]","[0008, 0007]",0003,551ea23f-12d4-4e32-b1fb-4ebd5ba05946,0.025579
3,0001,E1,[0001],"[0001, 0002, 0003, 0005, 0008]","[0007, 0006]",0004,c7d39827-0b39-4afe-9a2c-6f4f40ef25db,0.019417
4,0001,E1,[0001],"[0001, 0003, 0006, 0007, 0008]","[0002, 0004]",0005,9dd371bc-8e64-4bfd-9989-63b703f890dd,0.017488
5,0001,E1,[0001],"[0001, 0002, 0004, 0005, 0008]","[0003, 0007]",0006,23a0ce88-30ed-4fdc-9ec5-50c6ea04fcfb,0.024179
6,0001,E1,[0001],"[0001, 0002, 0003, 0006, 0008]","[0004, 0005]",0007,369117df-e8c4-4beb-b47e-5fda47d9a8dc,0.020952
7,0001,E1,[0001],"[0001, 0002, 0004, 0005, 0006]","[0003, 0007]",0008,d59f29db-9691-415f-9934-421863857d6f,0.025579
8,0002,E2,[0002],"[0002, 0003, 0005, 0006, 0007]","[0008, 0004]",0001,eb49479e-c1aa-4443-a3ac-4d5e72cdc3d4,0.029889
9,0002,E2,[0002],"[0003, 0005, 0006, 0007, 0008]","[0001, 0004]",0002,4ba9282f-255e-4aff-bee3-1e14c48ba23a,0.024111


## save results

In [22]:
#create parent folder
experimentpath=datapath+"/experiments/"+experimentId
os.mkdir(experimentpath)
#save each of the models
for id, model in zip(modelIds,models):
    model.save_model(experimentpath+"/"+id+".json")
#save experiment information
dumpPickle(experimentpath+"/experimentModels.pkl",experimentModels)
dumpPickle(experimentpath+"/featureSelection.pkl",featureSelection)

In [23]:
#you need to use this in the following parts of the experiment
print(experimentId)

77cdf395-3015-4c9e-a90b-a661d7715c16
